# Kernels CUDA personalizados em Python com Numba

Nesta seção, vamos aprofundar nosso entendimento de como o modelo de programação CUDA organiza o trabalho paralelo e aproveitar esse entendimento para escrever **kernels** CUDA, funções que são executadas em paralelo em GPUs CUDA. Os kernels CUDA personalizados, ao utilizar o modelo de programação CUDA, exigem mais trabalho para implementar do que, por exemplo, simplesmente decorar um ufunc com `@vectorize`. No entanto, eles possibilitam computação paralela em lugares onde ufuncs simplesmente não são capazes, e fornecem uma flexibilidade que pode levar ao mais alto nível de desempenho.

Esta seção contém três apêndices para aqueles que estão interessados no estudo futher: uma variedade de técnicas de depuração para auxiliar a programação de sua GPU, links para referências de programação CUDA e cobertura de geração de números aleatórios suportados Numba na GPU.

## Objetivos

Quando concluir esta seção, poderá:

* Escreva kernels CUDA personalizados em Python e inicie-os com uma configuração de execução.
* Utilize loops de passada de grade para trabalhar em paralelo em grandes conjuntos de dados e aproveitar a coalescência de memória.
* Use operações atômicas para evitar condições de corrida ao trabalhar em paralelo.

## A necessidade de kernels personalizados

Ufuncs são fantasticamente elegantes, e para qualquer operação escalar que deve ser realizada elemento sábio em dados, ufuncs são provavelmente a ferramenta certa para o trabalho.

Como você sabe, há muitas, se não mais, classes de problemas que não podem ser resolvidos aplicando a mesma função a cada elemento de um conjunto de dados. Considere, por exemplo, qualquer problema que exija acesso a mais de um elemento de uma estrutura de dados para calcular sua saída, como algoritmos de *stencil*, ou qualquer problema que não possa ser expresso por um valor de entrada para um mapeamento de valor de saída, como uma redução. Muitos desses problemas ainda são inerentemente paralelizáveis, mas não podem ser expressos por um ufunc.

Escrever kernels CUDA personalizados, embora mais desafiadores do que escrever ufuncs acelerados por GPU, fornece aos desenvolvedores uma tremenda flexibilidade para os tipos de funções que podem enviar para serem executados em paralelo na GPU. Além disso, como você vai começar a aprender nesta e na próxima seção, ele também fornece controle refinado sobre *como* o paralelismo é conduzido expondo a hierarquia de threads do CUDA para desenvolvedores explicitamente.

Embora permanecendo puramente em Python, a maneira como escrevemos kernels CUDA usando Numba é muito similar de como os desenvolvedores desenvolvem em CUDA C/ C ++. Para aqueles que estão familiarizados com programação em CUDA C/ C ++, você provavelmente vai pegar kernels personalizados em Python com Numba muito rapidamente, e para programadores de linguagem paralela de primeira viagem, sei que o trabalho aqui irá ajudar bem se precisar ou desejar se aprofundar em CUDA em C/C++, ou 

e para aqueles de vocês aprendê-los pela primeira vez, Sei que o trabalho que você faz aqui também irá atendê-lo bem se você precisar ou desejar desenvolver CUDA em C/C++, ou mesmo, fazer um estudo da riqueza de recursos CUDA na web que são mais comumente retratando CUDA C/C++ código.


## Introdução aos Kernels CUDA

Ao programar no CUDA, os desenvolvedores escrevem funções para a GPU chamadas **kernels**, que são executadas, ou em linguagem CUDA, **launched**, nos muitos núcleos da GPU em  **threads** em paralelo. Quando os kernels são iniciados, os programadores usam uma sintaxe especial, chamada de configuração **execution** (também chamada de configuração de inicialização) para descrever a configuração da execução paralela.

Os slides a seguir (que aparecerão após a execução da célula abaixo) fornecem uma introdução de alto nível sobre como os kernels CUDA podem ser criados para trabalhar em grandes conjuntos de dados em paralelo no dispositivo GPU. Trabalhe nos slides e, em seguida, você começará a escrever e executar seus próprios kernels CUDA personalizados, usando as ideias apresentadas nos slides.

<!-- #TODO: adicionar os slides da NVIDIA? -->
## O primeiro kernel CUDA

Vamos começar com um exemplo concreto e muito simples, reescrevendo nossa função de adição para matrizes NumPy 1D. Os kernels CUDA são compilados usando o decorador `numba.cuda.jit`. `numba.cuda.jit` não deve ser confundido com o decorador `numba.jit` que você já aprendeu que otimiza funções para a CPU.

Vamos começar com um exemplo muito simples para destacar algumas das sintaxes essenciais. Vale mencionar que essa função em particular poderia de fato ser escrita como um ufunc, mas nós a escolhemos aqui para manter o foco no aprendizado da sintaxe. Vamos prosseguir para funções mais adequadas para serem escritas como um kernel personalizado abaixo. Certifique-se de ler os comentários com cuidado, pois eles fornecem algumas informações importantes sobre o código.


In [ ]:
from numba import cuda

# Note que o uso de um array "out". Kernels CUDA escritos com @cuda.jit não retornam valores, 
# assim como suas contrapartes C. Além disso, não há nenhuma atribuição explicita com @cuda.jit.
#Diferente do que ocorria em numba.jit.

@cuda.jit
def add_kernel(x, y, out):
    
    # Os valores reais das seguintes variáveis fornecidas pelo CUDA para índices de thread e bloco,
    # como parâmetros de função, não são conhecidos até que o kernel é lançado.
    
    # Este cálculo fornece um índice de encadeamento exclusivo dentro de toda a grade (veja os slides acima para mais)
    idx = cuda.grid(1)          # 1 = thread grid de 1 dimensao, retorna um valor unico.
                                # Esta função conveniencia fornecida pelo Numba é quivalente a
                                # `cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x`

    # Esta thread fará o trabalho no elemento de dados com o mesmo índice que o seu próprio
    # Índice exclusivo na grade.
    out[idx] = x[idx] + y[idx]



In [ ]:
import numpy as np

n = 4096
x = np.arange(n).astype(np.int32) # [0...4095] no host
y = np.ones_like(x)               # [1...1] no host

d_x = cuda.to_device(x) # Copia do x no device
d_y = cuda.to_device(y) # Copia do y no device
d_out = cuda.device_array_like(d_x) # Similar ao np.array_like, Mas para arrays no device 

# Por causa de como escrevemos o kernel acima, precisamos ter um mapeamento de 1 thread 
# para um elemento de dados,

# portanto, definimos o número de threads na grade (128*32) para igual a n (4096).
threads_per_block = 128
blocks_per_grid = 32

In [ ]:
add_kernel[blocks_per_grid, threads_per_block](d_x, d_y, d_out)
cuda.synchronize()
print(d_out.copy_to_host()) # Should be [1...4096]

### Faça você mesmo: Ajustar o código

Faça as seguintes pequenas alterações no código acima para ver como isso afeta sua execução. Faça suposições sobre o que vai acontecer antes de executar o código:

* Diminuir a variável `threads_per_block` 
* Diminuir a variável `blocks_per_grid` 
* Aumentar as variáveis `threads_per_block` e/ou `blocks_per_grid
* Remover ou comentar a chamada `Cuda.synchronize()` 

### Resultados

No exemplo acima, como o kernel é escrito para que cada thread funcione em exatamente um elemento de dados, é essencial que o número de threads na grade seja igual ao número de elementos de dados.

Reduzindo **o número de threads na grade**, reduzindo o número de blocos e/ou reduzindo o número de threads por bloco, há elementos onde o trabalho é deixado de lado e, portanto, podemos ver na saída que os elementos no final do array `d_out` não tinham valores adicionados a ele. Se você editou a configuração de execução reduzindo o número de threads por bloco, então na verdade existem outros elementos através da matriz `d_out` que não foram processados.

**Aumentar o tamanho da grade** na verdade cria problemas com acesso à memória fora dos limites. Este erro não será exibido no seu código atualmente, mas mais adiante nesta seção você aprenderá como expor esse erro usando `Cuda-memcheck` e depurá-lo.

Você poderia esperar que **remover o ponto de sincronização** resultaria em uma impressão mostrando que nenhum ou menos trabalho foi feito. Este é um palpite razoável, pois sem um ponto de sincronização a CPU funcionará de forma assíncrona enquanto a GPU estiver processando. O detalhe a aprender aqui é que as cópias de memória carregam sincronização implícita, tornando a chamada para `Cuda.synchronize` acima desnecessária.

### Exercício: Acelere uma função de CPU como um kernel CUDA personalizado

Abaixo está a função escalar da CPU `square_device` que poderia ser usada como uma CPU ufunc. Seu trabalho é refatorá-lo para ser executado como um kernel CUDA com o `@Cuda.Jit`.

Você pode pensar que fazer esta função funcionar no dispositivo poderia ser muito mais facilmente feito com `@vectorize`, e você estaria correto. Mas esse cenário lhe dará a chance de trabalhar com toda a sintaxe que introduzimos antes de seguir para exemplos mais complicados e realistas.

Neste exercício, terá de:
* Refatorar a definição `square_device` para ser um kernel CUDA que fará o trabalho de um thread em um único elemento.
* Refazer os arrays `d` e `d_out` abaixo para serem arrays de dispositivos CUDA.
* Modificar as variáveis `blocks` e `threads` para valores apropriados para o fornecido `n`.
* Refatorar a chamada para `square_device` para ser um lançamento do kernel que inclui uma configuração de execução.

O teste de asserção abaixo falhará até que você implemente com sucesso o acima. Se você ficar preso, sinta-se livre para conferir a solução, logo abaixo.

<!-- #TODO: Coloco a resolução já aqui ? -->
### Exercício:  Faça você mesmo !

In [ ]:
@cuda.jit
def square_device(a):
    return a**2


# Deixe "n" fixo para este exercicio
n = 4096

a = np.arange(n)
out = a**2 # Apenas para criterio de comparação


d_a = a                  # faça uma função de envio p/ device I 
d_out = np.zeros_like(a) # faça uma função de envio p/ device II


# TODO: Atualize os valores de blocos e threads de acordo com o necessario
blocks = 0
threads = 0

# TODO: Launch as a kernel with an appropriate execution configuration
d_out = square_device(d_a)

In [ ]:
from numpy import testing
testing.assert_almost_equal(d_out, out)

### Solução - tente antes de visualizar !

In [ ]:
# Refactor to be a CUDA kernel doing one thread's work.
# Don't forget that when using `@cuda.jit`, you must provide an output array as no value will be returned.
def square_device(a):
    return a**2

# Leave the values in this cell fixed for this exercise
n = 4096

a = np.arange(n)
out = a**2

d_a = cuda.to_devide(a)             
d_out = cuda.device_array_like(d_a) 

# TODO: Update the execution configuration for the amount of work needed
blocks = 128
threads = 32

d_out = square_device(d_a)


## Um aparte em esconder opções de configuração de latência e execução

As GPUs NVIDIA habilitadas para CUDA consistem em vários multiprocessadores de streaming ou *SM's* em um dado, com DRAM conectado. *SM's* contêm todos os recursos necessários para a execução do código do kernel, incluindo muitos núcleos CUDA. Quando um kernel é lançado, cada bloco é atribuído a um único *SM*, com potencialmente muitos blocos atribuídos a um único *SM*. Blocos de partição *SM's* em subdivisões adicionais de 32 threads chamados warps e são esses warps que recebem instruções paralelas para executar.

Quando uma instrução leva mais de um ciclo de clock para ser concluída (ou na linguagem CUDA, para **expirar**), o *SM* pode continuar a fazer um trabalho significativo se ainda há warps que estão prontas para reber novas instruções. Por causa de arquivos de registro muito grandes no *SM's*, não há penalidade de tempo para um *SM* para alterar o contexto entre a emissão de instruções para uma warp ou outra. Em suma, a latência das operações pode ser ocultada por *SM's* com outro trabalho significativo, desde que haja outro trabalho a ser feito, de modo que a mesma SM seja reutilizada ao longo da mesma submissão do kernel.

Portanto, de importância primária para utilizar todo o potencial da GPU e, assim, escrever aplicativos acelerados de desempenho, é essencial dar ao *SM's* a capacidade de ocultar a latência, fornecendo-lhes um número suficiente de warps que podem ser realizadas mais simplesmente executando núcleos com dimensões de grade e bloco suficientemente grandes.



Decidir o melhor tamanho para a grade de thread CUDA é um problema complexo e depende tanto do algoritmo quanto da GPU específica [capacidade de computação] (https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-compute-capacidades), mas aqui estão alguns padrões que tendemos a seguir e que podem funcionar bem para começar:

  * O tamanho de um bloco deve ser um múltiplo de 32 threads (o tamanho de uma urdidura), com tamanhos de bloco típicos entre 128 e 512 threads por bloco.
  * O tamanho da grade deve garantir que a GPU completa seja utilizada sempre que possível. Lançar uma grade onde o número de blocos é 2x-4x o número de *SM's* na GPU é um bom ponto de partida. Algo na faixa de 20 - 100 blocos é geralmente um bom ponto de partida.
  * A sobrecarga de lançamento do kernel CUDA aumenta com o número de blocos, portanto, quando o tamanho da entrada é muito grande, achamos melhor não lançar uma grade em que o número de threads seja igual ao número de elementos de entrada, o que resultaria em um grande número de blocos. Em vez disso, usamos um padrão para o qual agora vamos voltar nossa atenção para lidar com grandes entradas.


## Trabalhando nos maiores conjuntos de dados com Grid Stride Loops

Os slides a seguir fornecem uma visão geral de alto nível de uma técnica chamada **grid stride loop** que criará kernels flexíveis onde cada thread é capaz de trabalhar em mais de um elemento de dados, uma técnica essencial para grandes conjuntos de dados. Execute a célula para carregar os slides.

<!-- #TODO: devo colocar os slides ??  -->
from IPython.display import IFrame
IFrame('https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-02-V1/AC_CUDA_Python_2.pptx', 640, 390)

## Um primeiro loop de Stride Grid

Vamos refatorar o `add_kernel` acima para utilizar um loop de stride Grid para que possamos lançá-lo para trabalhar em conjuntos de dados maiores de forma flexível, incorrendo nos benefícios da coalescência global de **memória**, que permite que threads paralelos acessem memória em partes contíguas, Um cenário que a GPU pode utilizar para reduzir o número total de operações de memória:

In [ ]:
from numba import cuda
import numpy as np

@cuda.jit
def add_kernel(x, y, out):
    

    start = cuda.grid(1) # `cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x`
    
    # This calculation gives the total number of threads in the entire grid
    stride = cuda.gridsize(1)   # 1 =  thread grid de uma dimensao, returns a single value.
                                # Conveniencia do Numba, que pode ser obtida por:
                                # `cuda.blockDim.x * cuda.gridDim.x`

  # Este thread começará a trabalhar no índice do elemento de dados igual ao seu próprio
  # índice exclusivo na grade e, em seguida, passará o número de threads na grade cada iteração, 
  # desde que não tenha saído dos limites dos dados. Dessa forma, cada encadeamento
  # pode funcionar em mais de um elemento de dados e, juntos, todos os encadeamentos 
  # funcionarão em todos os elementos de dados.

    for i in range(start, x.shape[0], stride):
        # Assuming x and y inputs are same length
        out[i] = x[i] + y[i]



n = 100000 # Esse valor é muito mais threads que temos no grid
x = np.arange(n).astype(np.int32)
y = np.ones_like(x)

d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_out = cuda.device_array_like(d_x)

threads_per_block = 128
blocks_per_grid = 30

In [ ]:
add_kernel[blocks_per_grid, threads_per_block](d_x, d_y, d_out)
print(d_out.copy_to_host()) # Remember, memory copy carries imp

### Exercício: Implementar um Loop de Stride Grid 

Refatore a seguinte função escalar `hypot_stride` da CPU para executar como um kernel CUDA utilizando um loop de Stride Grid. Sinta-se livre para olhar a solução, logo abaixo!

In [ ]:

from math import hypot
from numba import cuda
import numpy as np

def hypot_stride(a, b, c):
    c = hypot(a, b)


# Voce não precisa modificar nada neste bloco de comandos !
n = 1000000
a = np.random.uniform(-12, 12, n).astype(np.float32)
b = np.random.uniform(-12, 12, n).astype(np.float32)
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.device_array_like(d_b)

blocks = 128
threads_per_block = 64

hypot_stride[blocks, threads_per_block](d_a, d_b, d_c)


In [ ]:
# Teste de implementação, caso dê falha tente modificar algo faltante
from numpy import testing
testing.assert_almost_equal(np.hypot(a,b), d_c.copy_to_host(), decimal=5)

### Solução:

In [ ]:
from math import hypot

@cuda.jit
def hypot_stride(a, b, c):
        # Primeira maneira de Resolucao
    start = cuda.grid(1)
    stride = cuda.gridsize(1)
        # Segunda maneira de Resolucao
    # start  = cuda.threadIdx + cuda.blockDim.x * cuda.blockIdx.x
    # stride = cuda.blockDim.x * cuda.GridDim.x
    for i in range(start, a.shape[0], stride): 
        c[i] = hypot(a[i], b[i])

# Voce não precisa modificar nada neste bloco de comandos !
n = 1000000
a = np.random.uniform(-12, 12, n).astype(np.float32)
b = np.random.uniform(-12, 12, n).astype(np.float32)
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.device_array_like(d_b)

blocks = 128
threads_per_block = 64

hypot_stride[blocks, threads_per_block](d_a, d_b, d_c)

## Cronometrando o Kernel

Vamos aproveitar o tempo para fazer algum tempo de desempenho para o kernel `hypot_stride` . Se você não conseguiu implementá-lo com sucesso, copie e execute a solução.

## Linha de base da CPU

Primeiro vamos obter uma linha de base com `np.hypot`:

In [ ]:
%timeit np.hypot(a, b)

### Numba na CPU

Em seguida, vamos ver sobre uma versão otimizada para CPU:

In [ ]:
from numba import jit

@jit
def numba_hypot(a, b):
    return np.hypot(a, b)

In [ ]:
%timeit numba_hypot(a, b)

### Dispositivo Single Thread

Só para ver, vamos lançar nosso kernel em uma grade com apenas um único thread. Aqui usaremos `%time`, que só executa a instrução uma vez para garantir que nossa medição não seja afetada pela profundidade finita da fila do kernel CUDA. Iremos também adicionar um `Cuda.synchronize` para termos a certeza que não obtemos quaisquer tempos inócuos por causa do retorno do controlo à CPU, onde está o temporizador, antes do kernel terminar:

%time hypot_stride[1, 1](d_a, d_b, d_c); cuda.synchronize()

Espero que não seja uma grande surpresa que isso seja muito mais lento do que até mesmo a execução da CPU de linha de base.

### Paralelismo no Device

In [ ]:
%time hypot_stride[128, 64](d_a, d_b, d_c); cuda.synchronize()

## Operações Atômicas e Evitando Condições de Corrida

CUDA, como muitos frameworks de execução paralela de propósito geral, torna possível ter condições de corrida em seu código.  Uma condição de corrida no CUDA surge quando threads leem ou escrevem de um local de memória que pode ser modificado por outro thread independente. De um modo geral, você precisa se preocupar com:

 * riscos de leitura após gravação: um thread está lendo um local de memória ao mesmo tempo que outro thread pode estar escrevendo nele.
 * write-after-write (WAW) hazards: Dois threads estão gravando no mesmo local de memória, e apenas uma gravação será visível quando o kernel estiver completo.
 
Uma estratégia comum para evitar esses dois riscos é organizar o algoritmo do kernel CUDA de modo que cada encadeamento tenha responsabilidade exclusiva por subconjuntos exclusivos de elementos de matriz de saída e/ou nunca usar a mesma matriz para entrada e saída em uma única chamada do kernel. (Algoritmos iterativos podem usar uma estratégia de buffer duplo, se necessário, e alternar matrizes de entrada e saída em cada iteração.)

No entanto, há muitos casos em que diferentes threads precisam combinar resultados. Considere algo muito simples, como: "cada thread incrementa um contador global." A implementação disto no seu kernel requer que cada thread:

1. Leia o valor actual de um contador global.
2. Calcular `contador + 1`.
3. Escreva esse valor de volta à memória global.

No entanto, não há garantia de que outro encadeamento não tenha alterado o contador global entre as etapas 1 e 3. Para resolver esse problema, o CUDA fornece operações **atômicas** que lerão, modificarão e atualizarão um local de memória em uma etapa indivisível. Numba suporta várias dessas funções, [descritas aqui](http://numba.pydata.org/numba-doc/dev/cuda/intrinsics.html#supported-atomic-operations).

Vamos fazer o nosso kernel do contador de threads:

In [ ]:
@cuda.jit
def thread_counter_race_condition(global_counter):
    global_counter[0] += 1  # Isso é ruim !
    
@cuda.jit
def thread_counter_safe(global_counter):
    cuda.atomic.add(global_counter, 0, 1)  # Adiciona seguramente +1 no contador global

In [ ]:
# Devolve o resultado incorreto
global_counter = cuda.to_device(np.array([0], dtype=np.int32))
thread_counter_race_condition[64, 64](global_counter)

print('Should be %d:' % (64*64), global_counter.copy_to_host())

In [ ]:
# Funciona Corretamente
global_counter = cuda.to_device(np.array([0], dtype=np.int32))
thread_counter_safe[64, 64](global_counter)

print('Should be %d:' % (64*64), global_counter.copy_to_host())

### Desafio : Escreva um kernel de histograma acelerado !
Este exercício é considerado um desafio, pois será necessário fazer uso do que já aprendeu até aqui !
Tente elaborar este exercício levando em conta o exemplo (execução em CPU) na célula abaixo.

In [ ]:
def cpu_histogram(x, xmin, xmax, histogram_out):
  '''Incrementar contagens no histogram_out, dado intervalo de histograma [xmin, xmax).'''
    # Note que não temos que passar em nbins explicitamente, porque o tamanho do histogram_out determina isso
  nbins = histogram_out.shape[0]
  bin_width = (xmax - xmin) / nbins
    
  # Essa é uma maneira bem lenta de fazer isso com NumPy
  # mas parece similar com o que fará na GPU
  for element in x:
      bin_number = np.int32((element - xmin)/bin_width)
      if bin_number >= 0 and bin_number < histogram_out.shape[0]:
          # only increment if in range
          histogram_out[bin_number] += 1

In [ ]:
@cuda.jit
def cuda_histogram(x, xmin, xmax, histogram_out):
  #insira sua implementação aqui !

In [ ]:
d_x = cuda.to_device(x)
d_histogram_out = cuda.to_device(np.zeros(shape=10, dtype=np.int32))

blocks = 128
threads_per_block = 64

cuda_histogram[blocks, threads_per_block](d_x, xmin, xmax, d_histogram_out)

### Solução

In [ ]:
@cuda.jit
def cuda_histogram(x, xmin, xmax, histogram_out):
  nbins = histogram_out.shape[0]
  bin_width = (xmax - xmin) / nbins

  start = cuda.grid(1)
  stride = cuda.gridsize(1)
      # 2ª maneira - decl. das variaveis
  # start = cuda.threadIdx.x + cuda.blockDim.x * cuda.blockIdx.x
  # stride = cuda.gridDim.x * cuda.blockDim.x

  for element in range(start, nbins, stride):
      bin_number = np.int32((element - xmin)/bin_width)
      if bin_number >= 0 and bin_number < histogram_out.shape[0]:
        cuda.atomic.add(histogram_out, bin_number, 1)